In [1]:
from pathlib import Path
import pandas as pd
import elinker

CONFIG_FILE_PATH: /home/wkt406/miniconda3/envs/torch-env/lib/python3.10/site-packages/elinker/data/config.yml


# Read in Data

In [2]:
base_path = Path("../data/MedMentions")

In [3]:
med_mentions = elinker.datasets.med_mentions(base_path/"full/data/corpus_pubtator.txt")
doc_df = med_mentions[0]
mention_df = med_mentions[1]

In [4]:
mention_df[mention_df["sem_type_id"] == 'T191'].head()

,pmid,start_index,end_index,text_segment,sem_type_id,entity_id
764,26876532,49,70,parathyroid neoplasms,T191,C0030521
771,26876532,164,183,parathyroid adenoma,T191,C0262587
780,26876532,308,327,parathyroid adenoma,T191,C0262587
785,26876532,408,425,malignant tumours,T191,C0006826
789,26876532,527,545,parathyroid tumour,T191,C0030521


# Assign each PMID to dev, test, or training

In [5]:
with open(base_path/"full/data/corpus_pubtator_pmids_all.txt") as file:
    lines = file.readlines()
    all_pmids = [line.rstrip() for line in lines]
    
with open(base_path/"full/data/corpus_pubtator_pmids_dev.txt") as file:
    lines = file.readlines()
    dev_pmids = [line.rstrip() for line in lines]
    
with open(base_path/"full/data/corpus_pubtator_pmids_test.txt") as file:
    lines = file.readlines()
    test_pmids = [line.rstrip() for line in lines]
    
with open(base_path/"full/data/corpus_pubtator_pmids_trng.txt") as file:
    lines = file.readlines()
    train_pmids = [line.rstrip() for line in lines]

In [6]:
doc_df['split'] = None
doc_df.loc[doc_df.pmid.isin(train_pmids), 'split'] = 'train'
doc_df.loc[doc_df.pmid.isin(dev_pmids), 'split'] = 'dev'
doc_df.loc[doc_df.pmid.isin(test_pmids), 'split'] = 'test'

In [7]:
doc_df['split'].value_counts()

train    2635
test      879
dev       878
Name: split, dtype: int64

In [8]:
doc_df.to_parquet(base_path/"doc.parquet", index=False)
mention_df.to_parquet(base_path/"mention.parquet", index=False)

# Read Parquet Files

In [9]:
doc_df2 = pd.read_parquet(base_path/"doc.parquet")
mention_df2 = pd.read_parquet(base_path/"mention.parquet")

In [10]:
doc_df2.head()

,pmid,title,abstract,split
0,25763772,DCTN4 as a modifier of chronic Pseudomonas aer...,Pseudomonas aeruginosa (Pa) infection in cysti...,train
1,25847295,Nonylphenol diethoxylate inhibits apoptosis in...,Nonylphenol and short-chain nonylphenol ethoxy...,test
2,26316050,Prevascularized silicon membranes for the enha...,Recent advances in drug delivery and sensing d...,train
3,26406200,Seated maximum flexion: An alternative to stan...,The flexion - relaxation phenomenon (FRP) in s...,train
4,26424709,The Relationship Between Distance and Post-ope...,"To date, there is no research on voluntary med...",train


In [11]:
mention_df2.head()

,pmid,start_index,end_index,text_segment,sem_type_id,entity_id
0,25763772,0,5,DCTN4,"T116,T123",C4308010
1,25763772,23,63,chronic Pseudomonas aeruginosa infection,T047,C0854135
2,25763772,67,82,cystic fibrosis,T047,C0010674
3,25763772,83,120,Pseudomonas aeruginosa (Pa) infection,T047,C0854135
4,25763772,124,139,cystic fibrosis,T047,C0010674
